In [ ]:
# --- Imports (cell 1) ---
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional

import pandas as pd

from flovopy.asl.wrappers import run_single_event, find_event_files

In [ ]:
# --- Lightweight config object expected by run_single_event() (cell 2) ---
@dataclass
class LocalConfig:
    grid_kind: str = "regular"     # 'regular' or 'streams'
    wave_kind: str = "surface"     # 'surface' or 'body'
    dist_mode: str = "3d"          # '2d' or '3d'
    speed: float = 1.5             # km/s
    Q: int = 100
    misfit_engine: str = "l2"      # e.g. 'l2', 'huber'
    label: str = "nb_run"

    def tag(self) -> str:
        return f"{self.label}__G_{self.grid_kind}__W_{self.wave_kind}__D_{self.dist_mode}__V_{self.speed:g}__Q_{self.Q}__M_{self.misfit_engine}"

In [ ]:
# --- User-tunable paths & knobs (cell 3) ---

# Base project directories
PROJECTDIR = "/Users/GlennThompson/Dropbox/BRIEFCASE/SSADenver"
LOCALPROJECTDIR = "/Users/GlennThompson/work/PROJECTS/SSADenver_local"

# I/O
INPUT_DIR = f"{PROJECTDIR}/ASL_inputs/biggest_pdc_events"
OUTPUT_DIR = f"{LOCALPROJECTDIR}/asl_notebooks"
INVENTORY_XML = f"{PROJECTDIR}/MV.xml"

# Grid/data options
GRID_KIND = "streams"   # Montserrat: stream-based NodeGrid
CHANNELS_DIR = f"{PROJECTDIR}/channel_finder/channels_csv"
CHANNELS_STEP_M = 100.0
CHANNELS_DEM_TIF = f"{PROJECTDIR}/channel_finder/02_dem_flipped_horizontal.tif"
REGULAR_GRID_DEM = None   # not used here

# ASL parameters
PEAKF = 8.0
METRIC = "VT"
WINDOW_SECONDS = 5
NODE_SPACING_M = 50
MIN_STATIONS = 5
REFINE_SECTOR = False   # enable triangular dome-to-sea refinement

# Physics / attenuation
WAVE_KIND = "surface"   # surface waves
DIST_MODE = "3d"        # include elevation
SPEED = 1.5             # km/s
Q = 100
MISFIT_ENGINE = "l2"

# Basemap / region
DEM_TIF_FOR_BMAP = CHANNELS_DEM_TIF
REGION = (-62.255, -62.135, 16.66, 16.84)  # Montserrat default

# Limits
MAX_EVENTS: Optional[int] = 1  # cap number of events
GLOBAL_CACHE: Optional[str] = f"{LOCALPROJECTDIR}/asl_global_cache"

In [ ]:
# --- Build the config object (cell 4) ---
cfg = LocalConfig(
    grid_kind=GRID_KIND,
    wave_kind=WAVE_KIND,
    dist_mode=DIST_MODE,
    speed=SPEED,
    Q=Q,
    misfit_engine=MISFIT_ENGINE,
    label="notebook_run",
)
cfg.tag()

In [ ]:
# --- Discover event files (cell 5) ---
input_dir = Path(INPUT_DIR)
#event_files = list(find_event_files(str(input_dir)))
event_files = list(find_event_files(input_dir))
print(f"Found {len(event_files)} event files.")
if MAX_EVENTS is not None:
    event_files = event_files[:MAX_EVENTS]
    print(f"Limiting to {len(event_files)} files.")
#event_files[:5]
for f in event_files:
    print(f)

In [ ]:
# --- Run ASL per event (cell 6) ---
summaries: List[Dict[str, Any]] = []

for i, ev in enumerate(event_files, 1):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        cfg,
        mseed_file=str(ev),
        inventory_xml=str(INVENTORY_XML),
        output_base=str(OUTPUT_DIR),
        node_spacing_m=NODE_SPACING_M,
        metric=METRIC,
        window_seconds=WINDOW_SECONDS,
        peakf=PEAKF,
        channels_dir=CHANNELS_DIR,
        channels_step_m=CHANNELS_STEP_M,
        channels_dem_tif=CHANNELS_DEM_TIF,
        regular_grid_dem=REGULAR_GRID_DEM,
        dem_tif_for_bmap=DEM_TIF_FOR_BMAP,
        simple_basemap=True,
        refine_sector=REFINE_SECTOR,
        region=REGION,
        MIN_STATIONS=MIN_STATIONS,
        GLOBAL_CACHE=GLOBAL_CACHE,
    )
    summaries.append(result)

In [ ]:
# --- Summarize results (cell 7) ---
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")